In [2]:
# For now, I'm just focusing on delta hedging. After that is done, we make a system of equations
# to set delta and vega to 0 and solve with np.linalg.solve() (see examples/ex6sol for reference).
# %pip install pyfinance
import pandas as pd
from pyfinance.options import BSM
from datetime import datetime, timedelta


df = pd.read_csv("../data/apple.csv")
df = df.drop(["Unnamed: 0"], axis=1)
df['date'] = pd.to_datetime(df['Date'])
df['T'] = (pd.to_datetime('2024-01-19') - df['date']).dt.days / 365 #T = time to expiration in years
df = df.drop(["date"], axis=1)
#df.head()
df.info()

,Date,Underlying,C170,C175,C180,C185,C190,P170,P175,P180,P185,P190,T
0,2023-08-21,175.84,16.40,13.27,10.30,7.85,5.91,6.81,8.65,10.85,13.35,16.50,0.413699
1,2023-08-22,177.23,17.50,14.10,11.10,8.51,6.34,6.26,8.10,10.10,12.75,15.50,0.410959
2,2023-08-23,181.12,20.20,16.65,13.45,10.55,8.08,5.25,6.71,8.50,10.70,13.33,0.408219
3,2023-08-24,176.38,16.97,13.75,10.80,8.35,6.25,6.85,8.62,10.75,13.35,16.60,0.405479
4,2023-08-25,178.61,18.50,15.15,11.85,9.19,6.90,5.90,7.50,9.50,11.90,14.81,0.402740


The delta measures the rate of change of the security wrt. to the price of the underlying. Thus, the delta of the stock is one.
Vega measures the rate of change of the security wrt. to the volatility of the underlying. Again, the vega of the stock is one. In delta hedging, we short the option and hold a dynamically adjusted amount of the risky asset to eliminate the risk.

In [3]:
r = 0.01 # can be modified
sigma = 0.5 # calculate later, the volatility of the stock

def get_greeks(type, underlying, strike, T):
    # type is "call" or "put"
    val = BSM(kind=type, S0=underlying, K=strike, T=T, r=r, sigma=sigma).value()
    delta = BSM(kind=type, S0=underlying, K=strike, T=T, r=r, sigma=sigma).delta()
    vega = BSM(kind=type, S0=underlying, K=strike, T=T, r=r, sigma=sigma).vega()
    return val, delta, vega

print(get_greeks("call", 100, 95, 0.25)) #testing

(12.527923392521458, 0.633136941899257, 18.825749309685786)


Next, calculate the hedge with 2 portfolios: OP and RE. OP has a long call and RE has a short amount of the underlying asset (here, $AAPL). We want P = OP + RE to be neutral to small changes in the underlying price.

In [30]:
def delta_hedge(target_date):
    call_option_price = 0
    target_row = df[df['Date'] == target_date]

    if not target_row.empty:
        call_option_price = target_row['C185'].values[0]
        underlying = target_row['Underlying'][0]
        T = target_row['T'][0]
    else:
        print(f"No data found for the date {target_date}")
        return None

    print("ok")
    OP = call_option_price
    RE = - get_greeks("call", underlying, 185, T)[1] * underlying # replace again
    print("Aika =", target_date)
    print("Arvot", call_option_price, underlying, T)
    print("OP =", OP)
    print("RE =", RE)
    
    #next = """ we calculate changes in OP and RE and let A_0 = d OP - d RE"""

delta_hedge("2023-08-22")
print("")
#print(delta_hedge("2023-08-23"))

KeyError: 0